In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import requests
import re
from tqdm import tqdm
import time

In [ ]:
# Use reverse geocoding to use latitude and longitude to get the corresponding address components
api_key = ''
with open("geocoding_api_key.txt", 'r') as f:
    api_key = f.read()
    
f.close()

# REQUIRES: latlong coordinates for api lookup, api key
# MODIFIES: nothing 
# EFFECTS: queries google maps api to get location information
def get_address(latlong_coordinates, api_key, problem_rows, idx):
    street = None
    city = None
    state = None
    zipcode_5 = None
    
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    parameters = {'latlng':latlong_coordinates, 'key':api_key}
    
    try:
        response = requests.get(base_url, params=parameters)
        #print(response)
        full_address = response.json()['results'][0]['formatted_address']

        address_components = full_address.split(',')
        street = address_components[0]
        city = address_components[1]
        state_zip = address_components[2]

        # extract state and zipcode
        state = re.search('[A-Z]{2}', state_zip)[0]
        zipcode_5 = re.search('\\d{5}', state_zip)[0]

    except:
        print("Problem processing row ", idx, "- add to problem_rows list")
        problem_rows.append(idx)
        
    return street, city, state, zipcode_5, problem_rows

In [2]:
# File 1 (7,429 entries)
#Museums by Discipline ART, BOT, CMU, HST, NAT, SCI, and ZAW
f1 = pd.read_csv('MuseumFile2018_File1_Nulls.csv', encoding = "ISO-8859-1")
print(f1.shape)
print(f1.head())
print(f1.columns.get_values())

(7431, 57)
          MID DISCIPL        EIN  CO_LOC_RES NTEEC       DUNS   NAICS  \
0  8400100026     ART  631162561               A51  121013382  712110   
1  8400100029     ART  636049847               A51  075471300  712110   
2  8400100036     HST  631106693               A80  140201547  712110   
3  8400100037     BOT             ART MUSEUM                            
4  8400100045     ART                               804569858           

                                COMMONNAME  \
0                     MOBILE MUSEUM OF ART   
1           MONTGOMERY MUSEUM OF FINE ARTS   
2                 HISTORY MUSEUM OF MOBILE   
3  JASMINE HILL GARDENS AND OUTDOOR MUSEUM   
4                  JAN DEMPSEY ARTS CENTER   

                                    LEGALNAME ALTNAME  ...        GZIP  GZIP5  \
0                THE MOBILE MUSEUM OF ART INC          ...  36689-0426  36689   
1  MONTGOMERY MUSEUM OF FINE ARTS ASSOCIATION          ...       36117  36117   
2                   MOBILE MED

In [ ]:
# File 2 (7,959 entries): General Museums and Related Organizations GMU (i.e., general museums)
f2 = pd.read_csv('MuseumFile2018_File2_Nulls.csv', encoding = "ISO-8859-1")
print(f2.shape)
print(f2.head())

In [ ]:
# File 3 (14,783 entries): Historical Societies/Historic Preservation HSC
f3 = pd.read_csv('MuseumFile2018_File3_Nulls.csv', encoding = "ISO-8859-1")
print(f3.shape)
# has extra empty row (row 0)
f3.drop(0, inplace=True)
print(f3.shape)
print(f3.head())

In [ ]:
fig1 = f1['DISCIPL'].value_counts().plot(kind="bar")
fig1.set_xlabel("Discipline")
fig1.set_ylabel("Count")
fig1.set_title("Museum Survey Disciplines")

In [ ]:
# Check state distribution of museums
print(f1['PHSTATE'].value_counts())
fig2 = f1['PHSTATE'].value_counts().plot(kind="bar", figsize = (12,5))
fig2.set_xlabel("State")
fig2.set_ylabel("Count")
fig2.set_title("Museum Survey States Distribution")

In [ ]:
#print(f1['PHSTATE'].head())
#print(f1['PHZIP'].head())
print(f1.columns.get_values())
#print(f1['COMMONNAME'].head())
print(f1[['LEGALNAME','PHSTREET','PHCITY','PHSTATE','PHZIP','PHZIP5', 'INSTST','LONGITUDE','LATITUDE']].head())

In [ ]:
f_check = f1[f1['PHSTATE'] == ' ']
print(f_check.shape)
f_check[['LEGALNAME','PHSTREET','PHCITY','PHSTATE','PHZIP','PHZIP5', 'INSTST','LONGITUDE','LATITUDE']]


In [ ]:
problem_rows = []
for idx, row in tqdm(f1.iterrows()):
    if idx % 1000 == 0:
        print("idx is ", idx)
    
    latlong_coordinates =  row['LATITUDE'] + ',' + row['LONGITUDE']
    
    
    street, city, state, zipcode_5, problem_rows = get_address(latlong_coordinates, api_key, problem_rows, idx)
    
    f1.at[idx , 'PHSTREET'] = street
    f1.at[idx , 'PHCITY'] = street
    f1.at[idx , 'PHSTATE'] = state
    f1.at[idx , 'PHZIP5'] = zipcode_5
    

0it [00:00, ?it/s]

idx is  0


38it [00:12,  3.06it/s]

Problem processing row  38 - add to problem_rows list


76it [00:25,  3.05it/s]

Problem processing row  76 - add to problem_rows list


135it [00:43,  3.21it/s]

Problem processing row  135 - add to problem_rows list


276it [01:28,  3.25it/s]

In [4]:
f1.shape

(7431, 57)